In [1]:
import torch
torch.cuda.device_count()

16

In [2]:
from datasets import load_from_disk
import torch

### Preprocess the dataset

In [4]:
ds = load_from_disk("/data3/mmendieta/Violence_data/geo_corpus.0.0.1_datasets")

In [5]:
# Peek at one sample
ds["train"][0]

{'tweetid': 445511997619462144,
 'retweetid': 4.455081213779108e+17,
 'date': '2014-03-17',
 'timestamp': '2014-03-17 10:48:32',
 'username': 'miguelmorillo',
 'geo_x': -66.87919000000001,
 'geo_y': 10.488010000000001,
 'lang': 'es',
 'text': 'Venezuela en crisis, y la Fiscal de shopping en Alemania (Video)',
 'key': '-66_10__2014-03-17 10:48',
 'post1geo10': 0,
 'post1geo20': 0,
 'post1geo30': 0,
 'post1geo50': 0,
 'post1geo70': 0,
 'post2geo10': 0,
 'post2geo20': 0,
 'post2geo30': 0,
 'post2geo50': 0,
 'post2geo70': 0,
 'post3geo10': 0,
 'post3geo20': 0,
 'post3geo30': 0,
 'post3geo50': 0,
 'post3geo70': 0,
 'post7geo10': 1,
 'post7geo20': 1,
 'post7geo30': 1,
 'post7geo50': 1,
 'post7geo70': 1,
 'pre1geo10': 0,
 'pre1geo20': 0,
 'pre1geo30': 0,
 'pre1geo50': 0,
 'pre1geo70': 0,
 'pre2geo10': 0,
 'pre2geo20': 0,
 'pre2geo30': 0,
 'pre2geo50': 0,
 'pre2geo70': 0,
 'pre3geo10': 0,
 'pre3geo20': 0,
 'pre3geo30': 0,
 'pre3geo50': 0,
 'pre3geo70': 1,
 'pre7geo10': 0,
 'pre7geo20': 0,
 'pr

Since this is a multi-label classification problem, there are 40 labels = ('pre7geo10', 'pre7geo30', 'pre7geo50', 'post7geo10', 'post7geo30', 'post7geo50', ...)

In [6]:
# Remove unncesary columns
remove_cols = ['tweetid', 'retweetid', 'date', 'timestamp', 'username', 'geo_x', 'geo_y', 'lang', 'key']
remove_columns = [col for col in ds['train'].column_names if col in remove_cols]

In [7]:
ds = ds.remove_columns(remove_columns)

In [8]:
ds["train"].features

{'text': Value(dtype='string', id=None),
 'post1geo10': Value(dtype='int64', id=None),
 'post1geo20': Value(dtype='int64', id=None),
 'post1geo30': Value(dtype='int64', id=None),
 'post1geo50': Value(dtype='int64', id=None),
 'post1geo70': Value(dtype='int64', id=None),
 'post2geo10': Value(dtype='int64', id=None),
 'post2geo20': Value(dtype='int64', id=None),
 'post2geo30': Value(dtype='int64', id=None),
 'post2geo50': Value(dtype='int64', id=None),
 'post2geo70': Value(dtype='int64', id=None),
 'post3geo10': Value(dtype='int64', id=None),
 'post3geo20': Value(dtype='int64', id=None),
 'post3geo30': Value(dtype='int64', id=None),
 'post3geo50': Value(dtype='int64', id=None),
 'post3geo70': Value(dtype='int64', id=None),
 'post7geo10': Value(dtype='int64', id=None),
 'post7geo20': Value(dtype='int64', id=None),
 'post7geo30': Value(dtype='int64', id=None),
 'post7geo50': Value(dtype='int64', id=None),
 'post7geo70': Value(dtype='int64', id=None),
 'pre1geo10': Value(dtype='int64', id=N

In [9]:
# We need to cast integer labels to float in order to calculate the Binary Cross
# Entropy loss during training

from datasets import DatasetDict, Value

# List of label columns to cast (excluding 'text')
label_columns = [col for col in ds["train"].features if col != "text"]

# Automatically generate new features
new_features = ds["train"].features.copy()
for col in label_columns:
    new_features[col] = Value("float32")

# Apply cast to all splits
for split in ds.keys():
    ds[split] = ds[split].cast(new_features)

Casting the dataset:   0%|          | 0/1677 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/420 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/233 [00:00<?, ?ba/s]

In [19]:
# List of label columns to cast (excluding 'text')
label_columns = [col for col in ds["train"].features if col != "text"]
label_columns

['post1geo10',
 'post1geo20',
 'post1geo30',
 'post1geo50',
 'post1geo70',
 'post2geo10',
 'post2geo20',
 'post2geo30',
 'post2geo50',
 'post2geo70',
 'post3geo10',
 'post3geo20',
 'post3geo30',
 'post3geo50',
 'post3geo70',
 'post7geo10',
 'post7geo20',
 'post7geo30',
 'post7geo50',
 'post7geo70',
 'pre1geo10',
 'pre1geo20',
 'pre1geo30',
 'pre1geo50',
 'pre1geo70',
 'pre2geo10',
 'pre2geo20',
 'pre2geo30',
 'pre2geo50',
 'pre2geo70',
 'pre3geo10',
 'pre3geo20',
 'pre3geo30',
 'pre3geo50',
 'pre3geo70',
 'pre7geo10',
 'pre7geo20',
 'pre7geo30',
 'pre7geo50',
 'pre7geo70',
 'labels']

In [10]:
ds["train"].features

{'text': Value(dtype='string', id=None),
 'post1geo10': Value(dtype='float32', id=None),
 'post1geo20': Value(dtype='float32', id=None),
 'post1geo30': Value(dtype='float32', id=None),
 'post1geo50': Value(dtype='float32', id=None),
 'post1geo70': Value(dtype='float32', id=None),
 'post2geo10': Value(dtype='float32', id=None),
 'post2geo20': Value(dtype='float32', id=None),
 'post2geo30': Value(dtype='float32', id=None),
 'post2geo50': Value(dtype='float32', id=None),
 'post2geo70': Value(dtype='float32', id=None),
 'post3geo10': Value(dtype='float32', id=None),
 'post3geo20': Value(dtype='float32', id=None),
 'post3geo30': Value(dtype='float32', id=None),
 'post3geo50': Value(dtype='float32', id=None),
 'post3geo70': Value(dtype='float32', id=None),
 'post7geo10': Value(dtype='float32', id=None),
 'post7geo20': Value(dtype='float32', id=None),
 'post7geo30': Value(dtype='float32', id=None),
 'post7geo50': Value(dtype='float32', id=None),
 'post7geo70': Value(dtype='float32', id=None),

In [13]:
# This cell takes approximately 95 + 23 + 13 mins to run
# It is important that the labels are float in order to calculate Binary Cross Entropy loss
# create 'labels' columm
cols = ds["train"].column_names
ds = ds.map(lambda x : {"labels": [x[c] for c in cols if c != "text"]})
ds                                   

  0%|          | 0/16769932 [00:00<?, ?ex/s]

  0%|          | 0/4192483 [00:00<?, ?ex/s]

  0%|          | 0/2329158 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'post1geo10', 'post1geo20', 'post1geo30', 'post1geo50', 'post1geo70', 'post2geo10', 'post2geo20', 'post2geo30', 'post2geo50', 'post2geo70', 'post3geo10', 'post3geo20', 'post3geo30', 'post3geo50', 'post3geo70', 'post7geo10', 'post7geo20', 'post7geo30', 'post7geo50', 'post7geo70', 'pre1geo10', 'pre1geo20', 'pre1geo30', 'pre1geo50', 'pre1geo70', 'pre2geo10', 'pre2geo20', 'pre2geo30', 'pre2geo50', 'pre2geo70', 'pre3geo10', 'pre3geo20', 'pre3geo30', 'pre3geo50', 'pre3geo70', 'pre7geo10', 'pre7geo20', 'pre7geo30', 'pre7geo50', 'pre7geo70', 'labels'],
        num_rows: 16769932
    })
    validation: Dataset({
        features: ['text', 'post1geo10', 'post1geo20', 'post1geo30', 'post1geo50', 'post1geo70', 'post2geo10', 'post2geo20', 'post2geo30', 'post2geo50', 'post2geo70', 'post3geo10', 'post3geo20', 'post3geo30', 'post3geo50', 'post3geo70', 'post7geo10', 'post7geo20', 'post7geo30', 'post7geo50', 'post7geo70', 'pre1geo10', 'pre1ge

In [14]:
ds["train"][0]

{'text': 'Venezuela en crisis, y la Fiscal de shopping en Alemania (Video)',
 'post1geo10': 0.0,
 'post1geo20': 0.0,
 'post1geo30': 0.0,
 'post1geo50': 0.0,
 'post1geo70': 0.0,
 'post2geo10': 0.0,
 'post2geo20': 0.0,
 'post2geo30': 0.0,
 'post2geo50': 0.0,
 'post2geo70': 0.0,
 'post3geo10': 0.0,
 'post3geo20': 0.0,
 'post3geo30': 0.0,
 'post3geo50': 0.0,
 'post3geo70': 0.0,
 'post7geo10': 1.0,
 'post7geo20': 1.0,
 'post7geo30': 1.0,
 'post7geo50': 1.0,
 'post7geo70': 1.0,
 'pre1geo10': 0.0,
 'pre1geo20': 0.0,
 'pre1geo30': 0.0,
 'pre1geo50': 0.0,
 'pre1geo70': 0.0,
 'pre2geo10': 0.0,
 'pre2geo20': 0.0,
 'pre2geo30': 0.0,
 'pre2geo50': 0.0,
 'pre2geo70': 0.0,
 'pre3geo10': 0.0,
 'pre3geo20': 0.0,
 'pre3geo30': 0.0,
 'pre3geo50': 0.0,
 'pre3geo70': 1.0,
 'pre7geo10': 0.0,
 'pre7geo20': 0.0,
 'pre7geo30': 0.0,
 'pre7geo50': 0.0,
 'pre7geo70': 1.0,
 'labels': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,


In [15]:
# save the dataset only with columns 'text' and 'labels'
col_names = ds["train"].column_names
col_names.remove("labels")
col_names.remove('text')

In [16]:
ds_clean = ds.remove_columns(col_names)
ds_clean

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 16769932
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 4192483
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 2329158
    })
})

In [17]:
ds_clean.save_to_disk("/data3/mmendieta/Violence_data/geo_corpus.0.0.1_dataset_for_train_all_labels")